In [1]:
import pyspark

In [2]:
import pandas as pd

In [3]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as sf
appName= "hive_pyspark"
master= "local"

## create sparksession

In [4]:
from pyspark.sql import SparkSession

In [5]:
#spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [6]:
#spark

In [7]:
spark = SparkSession.builder.master(master).appName(appName).enableHiveSupport().getOrCreate()

22/03/23 12:00:39 WARN Utils: Your hostname, easebuzz-Inspiron-3501 resolves to a loopback address: 127.0.1.1; using 10.10.2.6 instead (on interface wlp0s20f3)
22/03/23 12:00:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/23 12:00:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df=spark.sql("show databases")
df.show()

22/03/23 12:00:43 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/03/23 12:00:43 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/03/23 12:00:46 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/03/23 12:00:46 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore easebuzz@127.0.1.1


+---------+
|namespace|
+---------+
|  default|
+---------+



In [9]:
datafile=spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Rbl_processed.csv",header=True)
datafile.show(5)
#datafile.write.saveAsTable("test_table")

+----------------+----------+-----------------------+------------+---------+----------+--------------+---------+--------------------+--------------------+
|Transaction Date|Value Date|Transaction Discription|Referance No|Debit Amt|Credit Amt|   Running Bal|Bank-name|           file-name|       date_ingested|
+----------------+----------+-----------------------+------------+---------+----------+--------------+---------+--------------------+--------------------+
|01-12-2021 06:10|01-12-2021|   NEFT/000214522423...|        null|     3.72|      null|2,47,97,607.12| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522442...|        null|     7.46|      null|2,47,97,599.66| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522424...|        null|     11.2|      null|2,47,97,588.46| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522466...|        null| 3,0

In [10]:
#converting dataframe perticular column to a list
list1=([data[0] for data in datafile.select('Transaction Discription').collect()])

In [11]:

df2 = datafile.where(
    datafile['Transaction Discription'].rlike("|".join(["(" + pat + ")" for pat in list1]))
)

In [12]:
df2.show()

+----------------+----------+-----------------------+------------+-----------+----------+--------------+---------+--------------------+--------------------+
|Transaction Date|Value Date|Transaction Discription|Referance No|  Debit Amt|Credit Amt|   Running Bal|Bank-name|           file-name|       date_ingested|
+----------------+----------+-----------------------+------------+-----------+----------+--------------+---------+--------------------+--------------------+
|01-12-2021 06:10|01-12-2021|   NEFT/000214522423...|        null|       3.72|      null|2,47,97,607.12| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522442...|        null|       7.46|      null|2,47,97,599.66| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522424...|        null|       11.2|      null|2,47,97,588.46| RBL-Bank|s3://reconcilatio...|2022-03-11 19:17:...|
|01-12-2021 06:10|01-12-2021|   NEFT/000214522466...|     

In [13]:
df2.dtypes

[('Transaction Date', 'string'),
 ('Value Date', 'string'),
 ('Transaction Discription', 'string'),
 ('Referance No', 'string'),
 ('Debit Amt', 'string'),
 ('Credit Amt', 'string'),
 ('Running Bal', 'string'),
 ('Bank-name', 'string'),
 ('file-name', 'string'),
 ('date_ingested', 'string')]

# --------------------------------------------------------------------------------------------------------------------------------

In [69]:
dfregset= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/Source_regiment.csv",header=True)
dfregset.show()

+--------------------+--------+
|           Regiments|    Arms|
+--------------------+--------+
|      Assam regiment|infantry|
|     Garehwal rifles|infantry|
|   Rajputana rifles |infantry|
|     Rajput regiment|infantry|
|Maratha light inf...|infantry|
| Sikh light infantry|infantry|
|       Sikh regiment|infantry|
|       Para regiment|infantry|
| para special forces|infantry|
|     Signal regiment|    ENTC|
|       508 regiment |Mech-inf|
|     Madras Regiment|infantry|
|          Granediers|infantry|
+--------------------+--------+



In [15]:
dffilterset= spark.read.csv("/media/easebuzz/DATA/Code/raw_files/filter_reg.csv",header=True)
dffilterset.show()

+----------+
|filter_reg|
+----------+
|     Assam|
|  Garehwal|
|       508|
+----------+



In [16]:
fillist=([data[0] for data in dffilterset.select('filter_reg').collect()])
print(fillist)


['Assam', 'Garehwal', '508']


In [17]:
afterfil = dfregset.where(
    dfregset['Regiments'].rlike("|".join(["(" + pat + ")" for pat in fillist]))
)
afterfil.show()


+---------------+--------+
|      Regiments|    Arms|
+---------------+--------+
| Assam regiment|infantry|
|Garehwal rifles|infantry|
|  508 regiment |Mech-inf|
+---------------+--------+



In [18]:
dfregset.subtract(afterfil).show()

+--------------------+--------+
|           Regiments|    Arms|
+--------------------+--------+
|     Madras Regiment|infantry|
|     Signal regiment|    ENTC|
|          Granediers|infantry|
|   Rajputana rifles |infantry|
|       Sikh regiment|infantry|
|     Rajput regiment|infantry|
| Sikh light infantry|infantry|
| para special forces|infantry|
|Maratha light inf...|infantry|
|       Para regiment|infantry|
+--------------------+--------+



In [19]:
#dftest = dfregset.merge(afterfil, how = 'outer' ,indicator=True).loc[lambda x : x['merge']=='left_only']
#dftest = df1.merge(df2, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

In [20]:
#diff_df = pd.merge(dfregset, afterfil, how='outer', indicator='Exist')

#diff_df = diff_df.loc[diff_df['Exist'] != 'both']

#df_diff = pd.concat([dfregset],afterfil).drop_duplicates(keep=False)

In [21]:
#dfregset=pd.concat[afterfil]

In [22]:
receipt = pd.read_excel("/media/easebuzz/DATA/Code/raw_files/receipt-14-03-2022-yespart A FINAL.xls")
print(receipt.head)

<bound method NDFrame.head of                       Date                                        Description  \
0               14/03/2022                                              Bin-1   
1               14/03/2022                                              Bin-2   
2               14/03/2022                                              Bin-3   
3               14/03/2022                     UPI Transaction dtd 12/03/2022   
4               14/03/2022                     UPI Transaction dtd 11/03/2022   
5               14/03/2022                                             Yes NB   
6               14/03/2022                                              Yes-1   
7               14/03/2022                                              Yes-2   
8               14/03/2022                                              Yes-3   
9               14/03/2022                                              Yes-4   
10              14/03/2022                                              Yes-5  

# ------------------------------------------------------------------------------------------------------------------------

In [23]:
Yesstatements =spark.read.csv('/media/easebuzz/DATA/Code/Processed_files/bankstatements_test.csv',header=True)
Yesstatements.show()

+-------------------+--------------------+--------------------+
|   transaction_date|          value_date|            NOTATION|
+-------------------+--------------------+--------------------+
|2021-11-01 00:25:00|2021-10-30 18:30:...|R/UPI/13051309360...|
|2021-11-01 05:57:00|2021-10-31 18:30:...|R/UPI/13051309735...|
|2021-11-01 09:52:00|2021-10-31 18:30:...|RTGS/UTIBH2130521...|
|2021-11-01 12:29:00|2021-10-31 18:30:...|IMPS 130512021968...|
|2021-11-01 06:25:00|2021-10-31 18:30:...|R/UPI/13051309792...|
|2021-11-01 13:06:00|2021-10-31 18:30:...|UPI/130555786563/...|
|2021-11-01 15:16:00|2021-10-31 18:30:...|IMPS 130515581433...|
|2021-11-01 17:18:00|2021-10-31 18:30:...|    TR9E6B0CDD65724F|
|2021-11-01 18:08:00|2021-10-31 18:30:...|IMPS 130518852314...|
|2021-11-01 20:14:00|2021-10-31 18:30:...|NEFT/INDBN0111485...|
|2021-11-01 23:17:00|2021-10-31 18:30:...|IMPS 130523849594...|
|2021-11-02 11:31:00|2021-11-01 18:30:...|R-130611071368-RE...|
|2021-11-02 12:38:00|2021-11-01 18:30:..

In [24]:
Utr =spark.read.csv("/media/easebuzz/DATA/Code/Processed_files/UtR_test.csv",header=True)
Utr.show(5)

+----------------+
|             UTR|
+----------------+
|    130513093601|
|    130513097355|
|UTIBH21305219225|
|    130512021968|
|    130513097925|
+----------------+
only showing top 5 rows



In [25]:
Utrlist=([data[0] for data in Utr.select('UTR').collect()])
print(Utrlist)


['130513093601', '130513097355', 'UTIBH21305219225', '130512021968', '130513097925', '130555786563', '130515581433', '130518852314', 'INDBN01114855660', '130523849594', '130611071368', '130613047373', 'SBIN321305229549', 'IBKLH21305600499', 'ICICH21306149418', '130619650378', 'HDFCH21306191224', '130622355229', '130513093616', '130513094584', '130513096509', '130509029235', '130509372224', '130710474439', 'N307211701074022', '130716067880', '130719049699', '130719050613', 'UTIBH21307245061', '130718510506', 'R-130718043751-REVERSAL-DP', 'IMPS 130719939340 FROM JUSTDIAL']


In [26]:
filtered_list=[]
for ele in Utrlist:
    if ele != None:
        filtered_list.append(ele)
print(filtered_list)

['130513093601', '130513097355', 'UTIBH21305219225', '130512021968', '130513097925', '130555786563', '130515581433', '130518852314', 'INDBN01114855660', '130523849594', '130611071368', '130613047373', 'SBIN321305229549', 'IBKLH21305600499', 'ICICH21306149418', '130619650378', 'HDFCH21306191224', '130622355229', '130513093616', '130513094584', '130513096509', '130509029235', '130509372224', '130710474439', 'N307211701074022', '130716067880', '130719049699', '130719050613', 'UTIBH21307245061', '130718510506', 'R-130718043751-REVERSAL-DP', 'IMPS 130719939340 FROM JUSTDIAL']


In [39]:
afterfil = Yesstatements.where(
    Yesstatements['NOTATION'].rlike("|".join(["(" + pat + ")" for pat in filtered_list]))
)

In [28]:
unmatched=Yesstatements.subtract(afterfil)

In [29]:
afterfil= afterfil.withColumn('Status', sf.lit('Matched'))

In [30]:
unmatched= unmatched.withColumn('Status', sf.lit('Unmatched'))

In [31]:
unmatched.show()

+-------------------+--------------------+--------------------+---------+
|   transaction_date|          value_date|            NOTATION|   Status|
+-------------------+--------------------+--------------------+---------+
|2021-11-09 17:32:00|2021-11-08 18:30:...|RTGS/IDFBH2131355...|Unmatched|
|2021-11-19 15:53:00|2021-11-18 18:30:...|UPI/132377504949/...|Unmatched|
|2021-11-05 20:27:00|2021-11-04 18:30:...|IMPS 130920591776...|Unmatched|
|2021-11-13 19:11:00|2021-11-12 18:30:...|IMPS 131719977880...|Unmatched|
|2021-11-01 19:25:00|2021-10-31 18:30:...|IMPS 130519185896...|Unmatched|
|2021-11-06 18:36:00|2021-11-05 18:30:...|RTGS/BCBMH2131073...|Unmatched|
|2021-11-03 19:40:00|2021-11-02 18:30:...|R-130719068598-RE...|Unmatched|
|2021-11-11 20:34:00|2021-11-10 18:30:...|RTGS/IDFBH2131556...|Unmatched|
|2021-11-02 13:23:00|2021-11-01 18:30:...|IMPS 130613116845...|Unmatched|
|2021-11-15 12:20:00|2021-11-14 18:30:...|IMPS 131912819043...|Unmatched|
|2021-11-12 10:47:00|2021-11-11 18:30:

In [32]:
afterfil.show()

+-------------------+--------------------+--------------------+-------+
|   transaction_date|          value_date|            NOTATION| Status|
+-------------------+--------------------+--------------------+-------+
|2021-11-01 00:25:00|2021-10-30 18:30:...|R/UPI/13051309360...|Matched|
|2021-11-01 05:57:00|2021-10-31 18:30:...|R/UPI/13051309735...|Matched|
|2021-11-01 09:52:00|2021-10-31 18:30:...|RTGS/UTIBH2130521...|Matched|
|2021-11-01 12:29:00|2021-10-31 18:30:...|IMPS 130512021968...|Matched|
|2021-11-01 06:25:00|2021-10-31 18:30:...|R/UPI/13051309792...|Matched|
|2021-11-01 13:06:00|2021-10-31 18:30:...|UPI/130555786563/...|Matched|
|2021-11-01 15:16:00|2021-10-31 18:30:...|IMPS 130515581433...|Matched|
|2021-11-01 18:08:00|2021-10-31 18:30:...|IMPS 130518852314...|Matched|
|2021-11-01 20:14:00|2021-10-31 18:30:...|NEFT/INDBN0111485...|Matched|
|2021-11-01 23:17:00|2021-10-31 18:30:...|IMPS 130523849594...|Matched|
|2021-11-02 11:31:00|2021-11-01 18:30:...|R-130611071368-RE...|M

In [33]:
print((afterfil.count(), len(afterfil.columns)))

(32, 4)


In [34]:
print((unmatched.count(), len(unmatched.columns)))

(168, 4)


In [35]:
print((Yesstatements.count(), len(Yesstatements.columns)))

(200, 3)


# ---------------------------------------------------------------------------------------------------------------------------------

In [36]:
#Yesstatements.withColumn("MATCHING",col("NOTATION").rlike(filtered_list.mkString("|"))).show(truncate = false)

In [37]:
#Yesstatements.withColumn("contains_cat_or_dog",col("phrase").rlike(animals.mkString("|"))).show(truncate = false)

# ------------------------------------------------------------------------------------------------------------------------------

In [40]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [41]:
data2 = [("James","","Smith","36636","M",3000,3000),
    ("Michael","Rose","","40288","M",4000,4500),
    ("Robert","","Williams","42114","M",4000,4000),
    ("Maria","Anne","Jones","39192","F",4000,3500),
    ("Jen","Mary","Brown","","F",-1,4000)
  ]

In [52]:
schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True), \
    StructField("refnum",StringType(), True) \
  ])

In [53]:
testdf=spark.createDataFrame(data2,schema=schema)

In [55]:
testdf.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- refnum: string (nullable = true)



In [59]:
#testdf.show(truncate=False)